In [16]:
from typing import Iterable, List
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from timeit import default_timer as timer
from torch.nn import Transformer
from torch import Tensor
from sklearn.model_selection import train_test_split
import copy
import tqdm
import librosa
import seaborn as sns
import torch.nn as nn
import torch
import torch.nn.functional as F
import numpy as np
import math
import os
#!pip install textgrid
import textgrid
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
import plotly.graph_objects as go
import pandas as pd
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import nltk
nltk.download('cmudict')
from nltk.corpus import cmudict
from transformers import AutoProcessor, AutoModelForCTC
import soundfile as sf

#_ESPEAK_LIBRARY = r"C:\Program Files\eSpeak NG\libespeak-ng.dll"
#EspeakWrapper.set_library(_ESPEAK_LIBRARY)
'''processor_P = AutoProcessor.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")
model_P = AutoModelForCTC.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")'''


from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
processor_H = AutoProcessor.from_pretrained("facebook/hubert-large-ls960-ft")

# 加载HuBERT模型
model_H = AutoModelForCTC.from_pretrained("facebook/hubert-large-ls960-ft")

[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


c:\Users\Alex\anaconda3\envs\BayesPCN\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at facebook/hubert-large-ls960-ft were not used when initializing HubertForCTC: ['hubert.encoder.pos_conv_embed.conv.weight_g', 'hubert.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertForCTC 

In [ ]:
from rpy2.robjects.packages import importr
from rpy2.robjects import Formula, pandas2ri

import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

# Activate the pandas conversion
pandas2ri.activate()
#utils = importr('utils')
#utils.install_packages('lme4')


In [15]:
human_result_path=r"..\data\test.xlsx"
human_result = pd.read_excel(human_result_path)
human_result_1a=human_result[human_result["Experiment"]=="1a"]

In [13]:
def get_pathset(paths):
    return [os.path.join(dir, each_file) for dir, mid, files in os.walk(paths) for each_file in files if each_file.endswith(".wav")]
audio_dir =r"..\Nov10\speech_files"
set1_list=[0,1,2,3,4,5,6,7,8,9,10,12,13,14,15,16]
set2_list=[17,18,19,20,21,22,24,25,26,27,28,29,30,31,37,40]

In [17]:
import pickle
with open("..\O_Jan31\hubert_words_Transformer.pkl", "rb") as file:
    sentence_matrix1 = pickle.load(file)

In [18]:
def get_keywords_dict(human_result_1a):
    '''
    Return a dict, key: sentenceID, values: keywords
    '''
    keywords_dict={}
    for each_ in human_result_1a.values:
        sentenceID=each_[human_result_1a.columns.get_loc("SentenceID")]
        if sentenceID not in keywords_dict:
            keywords_dict[sentenceID]=[]
        keyword=each_[human_result_1a.columns.get_loc("Keyword")]
        if keyword not in keywords_dict[sentenceID]:
            keywords_dict[sentenceID].append(keyword)
    return dict(sorted(keywords_dict.items()))

def create_set(audio_dir, df, reduced_data):
    set1_list=[0,1,2,3,4,5,6,7,8,9,10,12,13,14,15,16]
    set2_list=[17,18,19,20,21,22,24,25,26,27,28,29,30,31,37,40]
    keywords_dict=get_keywords_dict(df)
    keywords=[j for i in list(keywords_dict.values()) for j in i]
    audio_path=get_pathset(audio_dir)[::-1]
    out_dict={}
    word_features=[[] for i in range(len(keywords))]
    for __, each_path in enumerate(audio_path):
        current_talker=os.path.basename(each_path)[:13]
        if current_talker not in out_dict.keys():
            out_dict[current_talker]=[[] for i in range(32)]
        tg = textgrid.TextGrid.fromFile(each_path[:-3]+"TextGrid")
        tg_sentence = tg[0]
        for _,i in enumerate(tg[0]):
            if i.mark!="":
                tg_sentence[_-1].maxTime=tg_sentence[_].minTime
        tg_sentence = [i for i in tg_sentence if i.mark!=""]
        tg_sentence=[tg_sentence[i] for i in set1_list+set2_list]
        tg_word = [i for i in tg[1] if i.mark!="" and i.mark!="sp"]
        count=0
        for _,each_sentence in enumerate(tg_sentence):
            sentence_total_length=each_sentence.maxTime-each_sentence.minTime
            for key_word in list(keywords_dict.values())[_]:

                for each_word_tg in tg_word:
                    if each_word_tg.mark.lower()==key_word:
                        if each_word_tg.minTime >= each_sentence.minTime and each_word_tg.maxTime <= each_sentence.maxTime:
                            start=each_word_tg.minTime
                            end=each_word_tg.maxTime
                            break

                word_cut_start=start-each_sentence.minTime
                word_cut_end=end-each_sentence.minTime
                word_start=round(reduced_data[_][__].shape[0]*word_cut_start/sentence_total_length)
                word_end=round(reduced_data[_][__].shape[0]*word_cut_end/sentence_total_length)

                features=copy.deepcopy(reduced_data[_][__][word_start:word_end,:])
                word_features[count].append(features)
                out_dict[current_talker][_].append(features)
                count+=1
    return word_features,out_dict
word_features,out_dict=create_set(audio_dir, human_result_1a, sentence_matrix1)

In [123]:
rpy2.__version__

'3.5.16'

In [ ]:
def get_keywords_dict(human_result_1a):
    '''
    Return a dict, key: sentenceID, values: keywords
    '''
    keywords_dict={}
    for each_ in human_result_1a.values:
        sentenceID=each_[human_result_1a.columns.get_loc("SentenceID")]
        if sentenceID not in keywords_dict:
            keywords_dict[sentenceID]=[]
        keyword=each_[human_result_1a.columns.get_loc("Keyword")]
        if keyword not in keywords_dict[sentenceID]:
            keywords_dict[sentenceID].append(keyword)
    return dict(sorted(keywords_dict.items()))

def create_set(audio_dir, df, reduced_data):
    set1_list=[0,1,2,3,4,5,6,7,8,9,10,12,13,14,15,16]
    set2_list=[17,18,19,20,21,22,24,25,26,27,28,29,30,31,37,40]
    keywords_dict=get_keywords_dict(df)
    keywords=[j for i in list(keywords_dict.values()) for j in i]

    audio_path=get_pathset(audio_dir)[::-1]
    out_dict={}

    word_features=[[] for i in range(len(keywords))]

    for __, each_path in enumerate(audio_path):
        #audio, sr = librosa.load(each_path)
        #wave_res = librosa.resample(audio, orig_sr=sr, target_sr=16000)

        current_talker=os.path.basename(each_path)[:13]
        if current_talker not in out_dict.keys():
            out_dict[current_talker]=[[] for i in range(32)]
        tg = textgrid.TextGrid.fromFile(each_path[:-3]+"TextGrid")
        tg_sentence = tg[0]
        for _,i in enumerate(tg[0]):
            if i.mark!="":
                tg_sentence[_-1].maxTime=tg_sentence[_].minTime
        tg_sentence = [i for i in tg_sentence if i.mark!=""]
        tg_sentence=[tg_sentence[i] for i in set1_list+set2_list]
        tg_word = [i for i in tg[1] if i.mark!="" and i.mark!="sp"]
        count=0
        for _,each_sentence in enumerate(tg_sentence):
            sentence_total_length=each_sentence.maxTime-each_sentence.minTime
            for key_word in list(keywords_dict.values())[_]:

                for each_word_tg in tg_word:
                    if each_word_tg.mark.lower()==key_word:
                        if each_word_tg.minTime >= each_sentence.minTime and each_word_tg.maxTime <= each_sentence.maxTime:
                            start=each_word_tg.minTime
                            end=each_word_tg.maxTime
                            break

                word_cut_start=start-each_sentence.minTime
                word_cut_end=end-each_sentence.minTime
                word_start=round(reduced_data[_][__].shape[0]*word_cut_start/sentence_total_length)
                word_end=round(reduced_data[_][__].shape[0]*word_cut_end/sentence_total_length)

                features=copy.deepcopy(reduced_data[_][__][word_start:word_end,:])
                word_features[count].append(features)
                out_dict[current_talker][_].append(features)
                count+=1


    return word_features,out_dict
word_features,out_dict=create_set(audio_dir, human_result_1a, sentence_matrix1)

def get_training_paths(TrainingTalkerID):
    path_list=[]
    TalkerID=[]
    for each_ID in TrainingTalkerID.split(", "):
        if each_ID[:3]=="CMN":
            TalkerID.append(f"ALL_{each_ID[-3:]}_M_CMN")
        else:
            TalkerID.append(f"ALL_{each_ID[-3:]}_M_ENG")
    return TalkerID

def get_keywords_list(df):
    out_dict={}
    for each_ in human_result_1a.values:
        keyword_loc=df.columns.get_loc("Keyword")
        key_word = each_[keyword_loc]
        sentenceID = each_[df.columns.get_loc("SentenceID")]
        if sentenceID not in out_dict.keys():
            out_dict[sentenceID]=[]
        if key_word not in out_dict[sentenceID]:
            out_dict[sentenceID].append(key_word)
    return out_dict

def get_exposure_set(feature_dict,trainingTalkerID,sentenceID,key_word):
    keywors_list = get_keywords_list(human_result_1a)
    set1_list=[0,1,2,3,4,5,6,7,8,9,10,12,13,14,15,16]
    set2_list=[17,18,19,20,21,22,24,25,26,27,28,29,30,31,37,40]
    features=[]
    for i in trainingTalkerID:
        sentence_ind=(set1_list+set2_list).index(int(sentenceID[-3:])-1)
        key_word_ind=keywors_list[sentenceID].index(key_word)
        features.append(copy.deepcopy(feature_dict[i][sentence_ind][key_word_ind]))
    return features

def get_test_feature(feature_dict, test_talker, sentenceID, key_word):
    keywors_list = get_keywords_list(human_result_1a)
    set1_list=[0,1,2,3,4,5,6,7,8,9,10,12,13,14,15,16]
    set2_list=[17,18,19,20,21,22,24,25,26,27,28,29,30,31,37,40]
    #print(sentenceID)
    sentence_ind=(set1_list+set2_list).index(int(sentenceID[-3:])-1)
    key_word_ind=keywors_list[sentenceID].index(key_word)

    return copy.deepcopy(feature_dict[test_talker][sentence_ind][key_word_ind])


from numba import njit
@njit
def weighted_minkowski(vec1, vec2,  tau, w=1):

    total = 0.0
    for m in range(len(vec1)):
        diff = w*abs(vec1[m] - vec2[m])
        total += (diff ** tau)
    return total**(1/tau)#np.sqrt(total)

@njit
def dtw_sim(seq1, seq2,  tau, k):
    n, m = len(seq1), len(seq2)
    dtw_matrix = np.full((n+1, m+1), np.inf)
    dtw_matrix[0, 0] = 0.0

    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = weighted_minkowski(seq1[i-1], seq2[j-1], tau)
            dtw_matrix[i, j] = cost + min(dtw_matrix[i-1, j],    # insertion
                                         dtw_matrix[i, j-1],    # deletion
                                         dtw_matrix[i-1, j-1])  # match
    return np.exp(-(dtw_matrix[n, m]/((n+m)/2))*k)# change to *k,

def sim_measure1(df, feature_dict,tau, k):
    sim_min_list=[]
    sim_std_list=[]
    sim_median_list=[]
    sim_mean_list=[]

    train_set_dict={}
    test_word_dict={}
    count=0
    error=0


    train_talker_list=[]
    test_talker_list=[]
    out_df=pd.DataFrame(columns=['Condition2', 'TrainingTalkerID', 'TestTalkerID','SentenceID', 'Keyword',  'distance_min', 'IsCorrect'])#'trial',
    for each_ in tqdm.tqdm(df.values):
        filename_loc=df.columns.get_loc("Filename")
        keyword_loc=df.columns.get_loc("Keyword")
        sentence_loc= df.columns.get_loc("SentenceID")
        training_talker_loc=df.columns.get_loc("TrainingTalkerID")

        set1_list=[0,1,2,3,4,5,6,7,8,9,10,12,13,14,15,16]
        set2_list=[17,18,19,20,21,22,24,25,26,27,28,29,30,31,37,40]
        if each_[df.columns.get_loc("TrainingTestSet")] == "set2,set1":
            train_set=set2_list
            test_set=set1_list
        else:
            train_set=set1_list
            test_set=set2_list

        test_file = [os.path.basename(each_[df.columns.get_loc("Filename")])[:13]]
        key_word = each_[keyword_loc] #string
        TrainingTalkerID = each_[training_talker_loc] #list of string
        sentenceID = each_[df.columns.get_loc("SentenceID")]
        trainingTalkerID=get_training_paths(TrainingTalkerID)

        train_talker_key=",".join(sorted(trainingTalkerID))
        if train_talker_key not in train_set_dict:
            train_set_dict[train_talker_key]={}
        if sentenceID not in train_set_dict[train_talker_key]:
            train_set_dict[train_talker_key][sentenceID]={}
        if key_word not in train_set_dict[train_talker_key][sentenceID]:

            training_features=get_exposure_set(feature_dict,trainingTalkerID,sentenceID,key_word)
            train_set_dict[train_talker_key][sentenceID][key_word]=copy.deepcopy(training_features)
        else:
            training_features=train_set_dict[train_talker_key][sentenceID][key_word]

        if test_file[0] not in test_word_dict:
            test_word_dict[test_file[0]]={}
        if sentenceID not in test_word_dict[test_file[0]]:
            test_word_dict[test_file[0]][sentenceID]={}
        if key_word not in test_word_dict[test_file[0]][sentenceID]:
            test_feature = get_test_feature(feature_dict, test_file[0], sentenceID, key_word)
            test_word_dict[test_file[0]][sentenceID][key_word]=copy.deepcopy(test_feature)
        else:
            test_feature = test_word_dict[test_file[0]][sentenceID][key_word]

        train_talker_list.append(",".join(trainingTalkerID))
        test_talker_list.append(test_file[0])
        sims=[]

        for _, each_train_feature in enumerate(training_features):
            #print(each_train_feature)
            #print(test_feature)
            X=each_train_feature#.transpose()
            Y=test_feature#.transpose()
            #D, wp = librosa.sequence.dtw(X, Y, metric='euclidean')
            #combined_length = X.shape[1] + Y.shape[1]
            #normalized_distance = D[-1, -1] / combined_length
            normalized_distance = dtw_sim(X, Y, tau, k)
            sims.append(normalized_distance)

            #out_df=pd.DataFrame(columns=['Condition2', 'TrainingTalkerID', 'TestTalkerID', 'Keyword','distance_min', 'IsCorrect'])
        out_df.loc[len(out_df)]=[each_[df.columns.get_loc("Condition2")], each_[df.columns.get_loc("TrainingTalkerID")],#trainingTalkerID[_],
                                    each_[df.columns.get_loc("TestTalkerID")], each_[sentence_loc],
                                    each_[df.columns.get_loc("Keyword")],
                                    #each_[df.columns.get_loc('trial')],
                                    np.max(sims),
                                    each_[df.columns.get_loc("IsCorrect")]]
    return out_df

def objective_function(params, out_dict, human_result_1a):

    tau, k = params
    print(tau, k)
    out_df=sim_measure1(human_result_1a, out_dict,tau, k)
    new_df=copy.deepcopy(out_df)
    new_df['similarity'] = new_df['distance_min'].apply(lambda x:np.max(x))
    
    TTID=new_df.columns.get_loc("TrainingTalkerID")
    new_TrainingTalkerID=[]
    for i in new_df.values:
        new_TrainingTalkerID.append( ",".join(sorted(i[TTID].split(", "))) )
    new_df["TrainingTalkerID1"]=new_TrainingTalkerID
    new_df['trial'] = (new_df.groupby(['Keyword', 'Condition2', 'TrainingTalkerID1', 'TestTalkerID', 'SentenceID'], as_index=False).ngroup() + 1)


    new_df2=new_df.groupby(['Keyword', 'Condition2', 'TrainingTalkerID1', 'TestTalkerID', 'SentenceID','trial'], as_index=False).agg( #'sentenceID'
        IsCorrect=('IsCorrect', 'mean'),
        #distance=('distance_min', 'min'),
        similarity=('similarity', 'mean'),
        numCorrect=('IsCorrect', lambda x: (x==1).sum()),
        numIncorrect=('IsCorrect', lambda x: (x==0).sum())
        )
    new_df2['similarity_scaled'] = (new_df2['similarity'] - np.mean(new_df2['similarity'])) / (2 * np.std(new_df2['similarity']))
    if np.std(new_df['similarity_scaled']) < 0.3:
        return {
                    "tau": [tau], 
                    "k": [k], 
                    "z-value": [0],  
                    "mean_sim": [float(np.mean(new_df2["similarity"]))], 
                    "sd_sim": [float(np.std(new_df2["similarity"]))],  
                    "error": ["std_sim<0.3"]  # 
                }
    else:
        try:
            import rpy2.robjects as ro
            import rpy2
            from rpy2.robjects import pandas2ri
            
            pandas2ri.activate()
            base = importr('base')
            stats = importr('stats')
            lme4 = importr('lme4')
            ro.r('options(warn=2)')
            #r_data = ro.conversion.py2rpy(new_df2)
            r_data = pandas2ri.py2rpy(new_df2)
            formula = Formula('cbind(numCorrect, numIncorrect) ~ 1 + similarity_scaled + (1 | SentenceID / Keyword) + (1| TestTalkerID)')
            glmerControl = lme4.glmerControl(optimizer="bobyqa", optCtrl=ro.vectors.ListVector({'maxfun': 1e6}))
        except Exception as e:
            return {
                    "tau": [tau], 
                    "k": [k], 
                    "z-value": [0],  
                    "mean_sim": [float(np.mean(new_df2["similarity"]))], 
                    "sd_sim": [float(np.std(new_df2["similarity"]))],  
                    "error": [e]  # 
                }
        try:
            model = lme4.glmer(formula, data=r_data, control=glmerControl, family=stats.binomial(link="logit"))
            #log_likelihood = ro.r['logLik'](model)
            summary = base.summary(model)
            coefficients = summary.rx2('coefficients')
            #z_value = coefficients.rx('similarity_scaled', 'z value')[0]
            z_value=coefficients[1][2]
            return {
                    "tau": [tau], 
                    "k": [k], 
                    "z-value": [z_value],  
                    "mean_sim": [float(np.mean(new_df2["similarity"]))], 
                    "sd_sim": [float(np.std(new_df2["similarity"]))],  
                    "error": ["NA"]  # 
                }
        except Exception as e:
            return {
                    "tau": [tau], 
                    "k": [k], 
                    "z-value": [0],  
                    "mean_sim": [float(np.mean(new_df2["similarity"]))], 
                    "sd_sim": [float(np.std(new_df2["similarity"]))],  
                    "error": [str(e)]  # 
                }
    

In [88]:
coefficients

array([[-8.36155072e+04,  4.12930011e+00, -2.02493171e+04,
         0.00000000e+00],
       [ 8.36176514e+04,  4.12581370e+00,  2.02669479e+04,
         0.00000000e+00]])

In [86]:
type(new_df2)

pandas.core.frame.DataFrame

In [96]:
tau_values = np.arange(1,11,1)
k_values = 10**np.arange(-4,1.1,0.1)
import itertools
import multiprocessing
import time
from joblib import Parallel, delayed
pandas2ri.activate()
params_grid=list(itertools.product(tau_values, k_values))

start_time = time.time()
n_jobs = min(32, multiprocessing.cpu_count())
results = Parallel(n_jobs=n_jobs)(delayed(objective_function)(_,out_dict, human_result_1a) for _ in params_grid)
end_time = time.time()


In [ ]:
results_df=pd.json_normalize(results)
results_df

results_df['tau'] = results_df['tau'].apply(lambda x: x[0] if isinstance(x, list) else x)
results_df['k'] = results_df['k'].apply(lambda x: x[0] if isinstance(x, list) else x)

results_df['z-value'] = results_df['z-value'].apply(lambda x: x[0] if isinstance(x, list) else x)
results_df['mean_sim'] = results_df['mean_sim'].apply(lambda x: x[0] if isinstance(x, list) else x)
results_df['sd_sim'] = results_df['sd_sim'].apply(lambda x: x[0] if isinstance(x, list) else x)
results_df['error'] = results_df['error'].apply(lambda x: x[0] if isinstance(x, list) else x)


,tau,k,z-value,mean_sim,sd_sim,error
0,1,0.000100,0.000000,0.999407,0.000617,std_sim<0.3
1,1,0.000126,0.000000,0.999253,0.000776,std_sim<0.3
2,1,0.000158,0.000000,0.999060,0.000977,std_sim<0.3
3,1,0.000200,0.000000,0.998817,0.001229,std_sim<0.3
4,1,0.000251,0.000000,0.998511,0.001546,std_sim<0.3
...,...,...,...,...,...,...
505,10,3.981072,9.740809,0.136836,0.328466,NA
506,10,5.011872,9.581618,0.131534,0.329290,NA
507,10,6.309573,9.472723,0.128370,0.329903,NA
508,10,7.943282,9.403990,0.126627,0.330292,NA


In [106]:
results_df.to_csv('Xie_Transformers.csv', index=False)

In [125]:
df = pd.json_normalize(results)


df['tau'] = df['tau'].apply(lambda x: x[0] if isinstance(x, list) else x)
df['k'] = df['k'].apply(lambda x: x[0] if isinstance(x, list) else x)
df['z-value'] = df['z-value'].apply(lambda x: x[0] if isinstance(x, list) else x)
df['mean_sim'] = df['mean_sim'].apply(lambda x: x[0] if isinstance(x, list) else x)
df['sd_sim'] = df['sd_sim'].apply(lambda x: x[0] if isinstance(x, list) else x)
tau_vals = tau_values  
k_vals = k_values
tau_grid, k_grid = np.meshgrid(tau_vals, k_vals)  


from scipy.interpolate import griddata
#z_grid = griddata((df['tau'], df['k']), df['z-value'], (tau_grid, k_grid), method='linear')

z_grid = griddata((df['tau'], df['k']), df['z-value'], (tau_grid, k_grid), method='nearest')

fig = go.Figure(data=[go.Surface(z=z_grid, x=tau_grid, y=k_grid, colorscale='Viridis')])

fig.update_layout(
    width=1000,
    height=1000,
    title='3D Surface Plot of output from 24th-Transformer-layer',
    scene=dict(
        xaxis_title='Tau',
        yaxis_title='K',
        zaxis_title='sd_sim',
        yaxis=dict(type='log')
    )
)

fig.show()

In [98]:

import pickle
with open("..\O_Jan31\hubert_words_CNN.pkl", "rb") as file:
    sentence_matrix2 = pickle.load(file)

word_features2,out_dict2=create_set(audio_dir, human_result_1a, sentence_matrix2)
tau_values = np.arange(1,11,1)
k_values = 10**np.arange(-4,1.1,0.1)
import itertools
import multiprocessing
import time
from joblib import Parallel, delayed
pandas2ri.activate()
params_grid=list(itertools.product(tau_values, k_values))

start_time = time.time()
n_jobs = min(32, multiprocessing.cpu_count())
results2 = Parallel(n_jobs=n_jobs)(delayed(objective_function)(_,out_dict2, human_result_1a) for _ in params_grid)
end_time = time.time()
print(f"Total wall time: {end_time - start_time:.2f} seconds")

Total wall time: 2084.34 seconds


In [116]:
df = pd.json_normalize(results2)


df['tau'] = df['tau'].apply(lambda x: x[0] if isinstance(x, list) else x)
df['k'] = df['k'].apply(lambda x: x[0] if isinstance(x, list) else x)
df['z-value'] = df['z-value'].apply(lambda x: x[0] if isinstance(x, list) else x)

tau_vals = tau_values  
k_vals = k_values
tau_grid, k_grid = np.meshgrid(tau_vals, k_vals)  


from scipy.interpolate import griddata
#z_grid = griddata((df['tau'], df['k']), df['z-value'], (tau_grid, k_grid), method='linear')

z_grid = griddata((df['tau'], df['k']), df['z-value'], (tau_grid, k_grid), method='nearest')

fig = go.Figure(data=[go.Surface(z=z_grid, x=tau_grid, y=k_grid, colorscale='Viridis')])

fig.update_layout(
    width=1000,
    height=1000,
    title='3D Surface Plot of output from 24th-Transformer-layer',
    scene=dict(
        xaxis_title='Tau',
        yaxis_title='K',
        zaxis_title='Z-value',
        yaxis=dict(type='log')
    )
)

fig.show()

In [118]:

import pickle
with open("..\O_Jan31\hubert_words_CNN.pkl", "rb") as file:
    sentence_matrix2 = pickle.load(file)

word_features2,out_dict2=create_set(audio_dir, human_result_1a, sentence_matrix2)
tau_values = np.arange(1,11,1)
k_values = 10**np.arange(-1,3.1,0.2)
import itertools
import multiprocessing
import time
from joblib import Parallel, delayed
pandas2ri.activate()
params_grid=list(itertools.product(tau_values, k_values))

start_time = time.time()
n_jobs = min(32, multiprocessing.cpu_count())
results3 = Parallel(n_jobs=n_jobs)(delayed(objective_function)(_,out_dict2, human_result_1a) for _ in params_grid)
end_time = time.time()
print(f"Total wall time: {end_time - start_time:.2f} seconds")

Total wall time: 911.62 seconds


In [124]:
df = pd.json_normalize(results3)


df['tau'] = df['tau'].apply(lambda x: x[0] if isinstance(x, list) else x)
df['k'] = df['k'].apply(lambda x: x[0] if isinstance(x, list) else x)
df['z-value'] = df['z-value'].apply(lambda x: x[0] if isinstance(x, list) else x)

tau_vals = tau_values
k_vals = k_values
tau_grid, k_grid = np.meshgrid(tau_vals, k_vals)


from scipy.interpolate import griddata
#z_grid = griddata((df['tau'], df['k']), df['z-value'], (tau_grid, k_grid), method='linear')

z_grid = griddata((df['tau'], df['k']), df['z-value'], (tau_grid, k_grid), method='nearest')

fig = go.Figure(data=[go.Surface(z=z_grid, x=tau_grid, y=k_grid, colorscale='Viridis')])

fig.update_layout(
    width=1000,
    height=1000,
    title='3D Surface Plot of output from 7th-CNN-layer',
    scene=dict(
        xaxis_title='Tau',
        yaxis_title='K',
        zaxis_title='Z-value',
        yaxis=dict(type='log')
    )
)

fig.show()

In [5]:
import os

os.environ['R_HOME'] = 'C:\\Program Files\\R\\R-4.4.1'
os.environ["PATH"] += os.pathsep + r"C:\Program Files\R\R-4.4.1\bin\x64"

print(os.environ.get("R_HOME"))

C:\Program Files\R\R-4.4.1


In [7]:
import rpy2.robjects as ro
import rpy2
print(rpy2.__version__)

3.5.16


In [ ]:
from joblib import Parallel, delayed
import time
import math
import multiprocessing

def cpu_stress(duration_sec=10):
    start = time.time()
    count = 0
    while time.time() - start < duration_sec:
        count += math.sqrt(count + 1)
    return duration_sec


n_jobs = min(30, multiprocessing.cpu_count())

start_time = time.time()
results = Parallel(n_jobs=n_jobs)(delayed(cpu_stress)(10) for _ in range(n_jobs))
end_time = time.time()

#print("\n".join(results))
print(f"Total wall time: {end_time - start_time:.2f} seconds")


Total wall time: 10.02 seconds


In [65]:
results

[10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10]